---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )

    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    data = pd.read_csv("university_towns.txt",sep='\n', header=None, names=['col'])
    data['State'] = [df.split('[')[0] if 'edit' in df else np.NaN for df in data['col']]
    data['State'].ffill(inplace=True)
    data['RegionName'] = [df.split(' (')[0] if 'edit' not in df else np.NaN for df in data['col']]
    data.drop(['col'], axis=1, inplace=True)
    data.dropna(inplace=True)

    return data

In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    data = pd.read_excel("gdplev.xls", skiprows=219, usecols=[4,6])
    data.columns=['Quarter','GDP']
    data['GDP Diff'] = data['GDP'].diff()
    dec = data[data['GDP Diff']<0]
    dec['index'] = dec.index
    dec['index diff'] = dec['index'].diff()
    minindex = dec['index diff'].idxmin()
    res = data['Quarter'].iloc[minindex-1]

    return res

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    data = pd.read_excel("gdplev.xls", skiprows=219, usecols=[4,6])
    data.columns=['Quarter','GDP']
    data['GDP Diff'] = data['GDP'].diff()
    dec = data[data['GDP Diff']>0]
    dec['index'] = dec.index
    dec['index diff'] = dec['index'].diff()
    maxindex = dec['index diff'].idxmax()
    res = data['Quarter'].iloc[maxindex+1]
       
    return res

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    data = pd.read_excel("gdplev.xls", skiprows=219, usecols=[4,6])
    data.columns=['Quarter','GDP']
    data['GDP Diff'] = data['GDP'].diff()
    dec=data[data['GDP Diff']<0]
    dec['index'] = dec.index
    dec['index diff'] = dec['index'].diff()
    dec.sort(ascending=False,inplace=True)
    minindex = dec['index diff'].idxmin()
    res = data['Quarter'].iloc[minindex]
    return res

In [7]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    data = pd.read_csv("City_Zhvi_AllHomes.csv")
    data.replace({'State':states},inplace=True)
    data.set_index(['State','RegionName'],inplace=True)
    Index = data.index
    data.reset_index(inplace=True)
    df = data.loc[:,'2000-01':'2016-08']

    s=[]

    for j in range(0,199,3):
        a=[]    
        for i in range(3):
            if j+i < 200:
                a.append(df.iloc[:,j+i])
        s.append(a)


    # len(s)
    # s
    l1=[]
    for i in range(0,len(s)):
        l1.append(pd.DataFrame(s[i]).mean())


    columns=['2000q1','2000q2','2000q3','2000q4','2001q1','2001q2','2001q3','2001q4','2002q1','2002q2','2002q3','2002q4',
                '2003q1','2003q2','2003q3','2003q4','2004q1','2004q2','2004q3','2004q4','2005q1','2005q2','2005q3','2005q4',
             '2006q1','2006q2','2006q3','2006q4',   
             '2007q1','2007q2','2007q3','2007q4','2008q1','2008q2','2008q3','2008q4','2009q1','2009q2','2009q3','2009q4',
                '2010q1','2010q2','2010q3','2010q4','2011q1','2011q2','2011q3','2011q4','2012q1','2012q2','2012q3','2012q4',
                '2013q1','2013q2','2013q3','2013q4','2014q1','2014q2','2014q3','2014q4','2015q1','2015q2','2015q3','2015q4',
                '2016q1','2016q2','2016q3']

    df = pd.DataFrame(np.transpose(l1))
    df.columns=columns
    df.index=Index

    return df

In [8]:
def run_ttest():

    """# First create new data showing the decline or growth of housing prices
    # between the recession start and the recession bottom. Then run a ttest
    # comparing the university town values to the non-university towns values, 
    # return whether the alternative hypothesis (that the two groups are the same)
    # is true or not as well as the p-value of the confidence. 

    # Return the tuple (different, p, better) where different=True if the t-test is
    # True at a p<0.01 (we reject the null hypothesis), or different=False if 
    # otherwise (we cannot reject the null hypothesis). The variable p should
    # be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    # value for better should be either "university town" or "non-university town"
    # depending on which has a lower mean price ratio (which is equivilent to a
    # reduced market loss)."""
    
    data = convert_housing_data_to_quarters()
    before_rec = (data.columns.get_loc(get_recession_start())-1)
    rec_bottom = data.columns.get_loc(get_recession_bottom())

    df = np.divide(data.iloc[:,before_rec], data.iloc[:,rec_bottom]).dropna()
    df=pd.DataFrame(df)
    uni = get_list_of_university_towns().set_index(['State','RegionName'])
    uni_towns = df.merge(uni, right_index=True,left_index=True,how='inner')
    non_uni_towns = df.drop(uni_towns.index)

    # A t-test is commonly used to determine whether the mean of a population significantly
    # differs from a specific value (called the hypothesized mean) or from the mean of another population.

    p_value = ttest_ind(uni_towns, non_uni_towns).pvalue

    if p_value < 0.01:
        different=True
    else:
        different=False


    if uni_towns.mean().values < non_uni_towns.mean().values:
        better='university town'
    else:
        better='non-university town'

    p = p_value[0]


    return (different, p, better)